<a href="https://colab.research.google.com/github/Haowjy/Discord-Bots/blob/main/Scrape_and_Train_Royalroad_Synopsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import re

In [ ]:
# the number of pages, 20 books per page
# synopsis top 5000

num_pages = int(5000/20)

synopsisFile = open("/content/drive/MyDrive/story scrape/Royalroad/royalroad_top_5000_synopsis.txt","w")
for i in range(1, num_pages+1):
    page = requests.get("https://www.royalroad.com/fictions/search?page="+str(i)+"&orderBy=popularity")
    soup = BeautifulSoup(page.content, 'html.parser')

    title_link_list = soup.find_all('a', class_="font-red-sunglo bold",href=True)
    descrip_list = soup.find_all('div', class_="margin-top-10 col-xs-12")
    for j in range(len(title_link_list[:])):
        book_title = title_link_list[j].text

        descrip = descrip_list[j].text

        synopsisFile.write("<|startoftext|>\n"+descrip+"<|endoftext|>\n")
synopsisFile.close()

GPT-2 Trained used [gpt-2-simple](https://github.com/minimaxir/gpt-2-simple)

In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


In [ ]:
# model = "124M"
modelName = "355M"
# model = "774M" # cannot be fine-tuned
gpt2.download_gpt2(model_name=modelName)

In [ ]:
%cp /content/drive/MyDrive/story scrape/Royalroad/royalroad_top_5000_synopsis.txt /content/royalroad_top_5000_synopsis.txt

In [ ]:
runName = 'royalroad_top_5000_synopsis'
#gpt2.copy_file_from_gdrive(runName+".txt")

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=runName+'.txt',
              model_name=modelName,
              steps=300,
              restore_from='fresh',
              run_name=runName,
              print_every=10,
              sample_every=100,
              save_every=100
              )

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name=runName)

In [ ]:
import tensorflow as tf
tf.reset_default_graph()
gpt2.copy_checkpoint_from_gdrive(run_name=runName)

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=runName)

In [ ]:
gpt2.generate(sess, run_name=runName, return_as_list=True, truncate="<|endoftext|>", nsamples=5, batch_size=5)